## Pseudo-code

In [47]:
# setup
# (x) randomly pick 300 words from ../vocab.txt (this will be given by Julia)
# (x) save that as a txt file, one line per word
# (x) generate a list containing 2500-3500ms (include start and end) in 250ms intervals
# (x) write a python function (inside) that 1) load 2) randomize and 3) return a list of 4 lists of 75 words
# (x) write a python function (inside) that randomly pick a interstimuli time

In [56]:
# experiment
# kNUM_WORDS_PER_COND = 75
# interstim_intervals = get_interstim_interval_for_all(size=300)
# for i, condition in enumerate(randomize(noise_conditions)):
#     word_list = word_lists[idx]
#     for j, word in enumerate(word_list):
#         print(word) <- timer starts
#         make noun judgment <- takes no time
#         record word presented, present time, interstimulus interval, response time <- takes no time
#         time.sleep(interstim_intervals[i * kNUM_WORDS_PER_COND + j])

In [49]:
import numpy as np

## Setup

In [61]:
kNUM_WORDS_PER_COND = 75
kNUM_CONDS = 4

Tasks:
- randomly pick 300 words from ../vocab.txt (this will be given by Julia)
- save that as a txt file, one line per word

In [50]:
with open('../vocab.txt', 'r') as txt_f:
    words = [word.rstrip() for word in txt_f.readlines()]

In [51]:
np.random.seed(1)
sdt_words = np.random.choice(words, size=300)

In [52]:
with open('sdt_vocab.txt', 'w+') as txt_f:
    for word in sdt_words:
        txt_f.write(word + '\n')

In [62]:
def get_word_lists(sdt_vocab_fpath):
    with open(sdt_vocab_fpath, 'r') as txt_f:
        words = [word.rstrip() for word in txt_f.readlines()]
    return np.split(np.array(words), kNUM_CONDS)

In [63]:
get_word_lists('sdt_vocab.txt')

[array(['ai', 'lips', 'berkeley', 'sanyo', 'education', 'leadership',
        'administered', 'holdings', 'running', 'finder', 'updating',
        'mambo', 'maintain', 'emergency', 'development', 'stamp', 'folder',
        'jay', 'shipments', 'performer', 'ash', 'significance', 'entering',
        'balloon', 'wearing', 'concept', 'brands', 'organizing', 'oak',
        'boc', 'implementing', 'eden', 'formed', 'vessels', 'differences',
        'th', 'polyphonic', 'anything', 'harley', 'arrivals', 'turbo',
        'chamber', 'capitol', 'pump', 'supposed', 'circles', 'error',
        'competitors', 'vienna', 'divide', 'abraham', 'cursor', 'sd',
        'campbell', 'wisconsin', 'boobs', 'enter', 'british', 'vegetable',
        'fifty', 'adopt', 'impacts', 'attitudes', 'vendors', 'retro',
        'clinics', 're', 'selected', 'hand', 'airfare', 'carroll',
        'bulletin', 'replace', 'latter', 'much'], dtype='<U13'),
 array(['longitude', 'jews', 'viagra', 'zip', 'eligible', 'mainstream',
  

Tasks:
- generate a list containing 2500-3500ms (include start and end) in 250ms intervals
- save that as a json file

In [53]:
def get_interstim_interval_for_all(size=300):
    choices = np.arange(2500, 3500 + 1, 250)
    return np.random.choice(choices, size=size)

In [54]:
get_interstim_interval_for_all()[:10]

array([3500, 3500, 3000, 3000, 3250, 2750, 3500, 2500, 2500, 3250])

In [55]:
len(get_interstim_interval_for_all())

300